In [1]:
import os
import re
import string
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import collections
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.python.framework import ops
ops.reset_default_graph()

C:\Users\Divyansh\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Now making a default graph session

In [2]:
sess = tf.Session()

Parameter : Descriptions

-    min_word_freq: Only attempt to model words that appear at least 5 times.
-    rnn_size: size of our RNN (equal to the embedding size)
-    epochs: Number of epochs to cycle through the data
-    batch_size: How many examples to train on at once
-    learning_rate: The learning rate or the convergence paramter
-    training_seq_len: The length of the surrounding word group (e.g. 10 = 5 on each side)
-    embedding_size: Must be equal to the rnn_size
-    save_every: How often to save the model
-   eval_every: How often to evaluate the model
-    prime_texts: List of test sentences


In [3]:
min_word_freqeuncy = 10
epochs = 10
batch_size = 100
rnn_size = 128
learning_rate = 0.001
training_seq_length = 50
embedding_size = rnn_size
save_every = 500
eval_every = 50
prime_texts = ['thou art more', 'to be or not to', 'wherefore art thou']

In [5]:
data_dir = 'temp' 
data_file = 'shakespeare.txt' 
model_path = 'shakespeare_model'
full_model_dir = os.path.join(data_dir, model_path)
# Declare punctuation to remove, everything except hyphens and apostrophe's , since shakespear uses them quite frequently 
# to join words .
punctuation = string.punctuation
punctuation = ''.join([x for x in punctuation if x not in ['-', "''"]])


# Make Model Directory
if not os.path.exists(full_model_dir):
    os.makedirs(full_model_dir)

# Make data directory
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
print('Loading Shakespeare Data')
# Check if file is downloaded.
if not os.path.isfile(os.path.join(data_dir, data_file)):
    print('Not found, downloading Shakespeare texts from www.gutenberg.org')
    shakespeare_url = 'http://www.gutenberg.org/cache/epub/100/pg100.txt'
    # Get Shakespeare text
    response = requests.get(shakespeare_url)
    shakespeare_file = response.content
    # Decode binary into string
    s_text = shakespeare_file.decode('utf-8')
    # Drop first few descriptive paragraphs.
    s_text = s_text[7675:]
    # Remove newlines
    s_text = s_text.replace('\r\n', '')
    s_text = s_text.replace('\n', '')
    
    # Write to file
    with open(os.path.join(data_dir, data_file), 'w') as out_conn:
        out_conn.write(s_text)
else:
    # If file has been saved, load from that file
    with open(os.path.join(data_dir, data_file), 'r') as file_conn:
        s_text = file_conn.read().replace('\n', '')

# Clean text
print('Cleaning Text')
s_text = re.sub(r'[{}]'.format(punctuation), ' ', s_text)
s_text = re.sub('\s+', ' ', s_text ).strip().lower()
print('Done loading/cleaning.')

Loading Shakespeare Data
Not found, downloading Shakespeare texts from www.gutenberg.org
Cleaning Text
Done loading/cleaning.
